![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft-small-logo.png)
 
## Deploy SQL Server 2019 Big Data Cluster on an existing cluster deployed using kubeadm
 
This notebook walks through the process of deploying a <a href="https://docs.microsoft.com/sql/big-data-cluster/big-data-cluster-overview?view=sqlallproducts-allversions">SQL Server 2019 Big Data Cluster</a> on an existing kubeadm cluster.

In [1]:
import json,sys,os
def run_command(command):
    print("Executing: " + command)
    !{command}
    if _exit_code != 0:
        sys.exit(f'Command execution failed with exit code: {str(_exit_code)}.\n\t{command}\n')
    print(f'Successfully executed: {command}')

### **Set variables**
Generated by Azure Data Studio using the values collected in the Deploy Big Data Cluster wizard

### **Set password**

In [3]:
mssql_password = os.environ["AZDATA_NB_VAR_BDC_ADMIN_PASSWORD"]
if mssql_auth_mode == "ad":
    mssql_domain_service_account_password = os.environ["AZDATA_NB_VAR_BDC_AD_DOMAIN_SVC_PASSWORD"]

### **Set and show current context**

In [0]:
run_command(f'kubectl config use-context {mssql_cluster_context}')
run_command('kubectl config current-context')

### **Create deployment configuration files**

In [6]:
mssql_target_profile = 'ads-bdc-custom-profile'
if not os.path.exists(mssql_target_profile):
    os.mkdir(mssql_target_profile)
bdcJsonObj = json.loads(bdc_json)
controlJsonObj = json.loads(control_json)
bdcJsonFile = open(f'{mssql_target_profile}/bdc.json', 'w')
bdcJsonFile.write(json.dumps(bdcJsonObj, indent = 4))
bdcJsonFile.close()
controlJsonFile = open(f'{mssql_target_profile}/control.json', 'w')
controlJsonFile.write(json.dumps(controlJsonObj, indent = 4))
controlJsonFile.close()
print(f'Created deployment configuration folder: {mssql_target_profile}')

### **Create SQL Server 2019 Big Data Cluster**

In [7]:
print (f'Creating SQL Server 2019 Big Data Cluster: {mssql_cluster_name} using configuration {mssql_target_profile}')
os.environ["ACCEPT_EULA"] = 'yes'
os.environ["AZDATA_USERNAME"] = mssql_username
os.environ["AZDATA_PASSWORD"] = mssql_password
if mssql_auth_mode == "ad":
    os.environ["DOMAIN_SERVICE_ACCOUNT_USERNAME"] = mssql_domain_service_account_username
    os.environ["DOMAIN_SERVICE_ACCOUNT_PASSWORD"] = mssql_domain_service_account_password
if os.name == 'nt':
    print(f'If you don\'t see output produced by azdata, you can run the following command in a terminal window to check the deployment status:\n\tkubectl get pods -n {mssql_cluster_name} ')
run_command(f'azdata bdc create -c {mssql_target_profile}')